<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/region_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install cityhash
import cityhash

from copy import deepcopy

     |████████████████████████████████| 51kB 2.3MB/s 
  Created wheel for cityhash: filename=cityhash-0.2.3.post9-cp36-cp36m-linux_x86_64.whl size=82874 sha256=89b815c1f1b655e74d6424b9740ea0eddb3ca6a392c5a6176e97710ffad869e3
  Stored in directory: /root/.cache/pip/wheels/a4/4d/ae/cf0f1f8058ca2738fe972c69291083a0a06765271fb1047edd
Successfully built cityhash


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
org_info = pd.read_csv('/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/organizations.csv')
print(np.shape(org_info))
# Filter down the data purely to investees
org_info = org_info[org_info['primary_role'] == 'company']
print(np.shape(org_info))

(1089733, 41)
(1025256, 41)


In [8]:
import collections

# Select organization data
# Clean the df

org_info_selected = org_info[['uuid', 'region']]
org_info_selected = org_info_selected.dropna()
org_info_selected = org_info_selected.reset_index(drop=True)
print("Rows after NA drop: ", np.shape(org_info_selected)[0])

regions = collections.Counter()

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  region = row['region']  
  regions[region] += 1

# Filter low count regions
low_count_threshold = 15
keys_to_remove = []
for region, count in regions.items():
  if count < low_count_threshold:
    keys_to_remove.append(region)

for key in keys_to_remove:
  del regions[key]

# Create index to word mapping
region_to_index = {}
# Start at 1 so we can put hashed uuid at 0.
region_index = 1
for region, count in regions.items():
  region_to_index[region] = region_index
  region_index += 1

print(region_to_index)
region_size = region_index

Rows after NA drop:  842699
Processing  842699  rows.
100000  of  842699
200000  of  842699
300000  of  842699
400000  of  842699
500000  of  842699
600000  of  842699
700000  of  842699
800000  of  842699
{'New York': 1, 'California': 2, 'Illinois': 3, 'Hong Kong Island': 4, 'Dublin': 5, 'Virginia': 6, 'Zurich': 7, 'Texas': 8, 'Florida': 9, 'Arizona': 10, 'Tel Aviv': 11, 'Connecticut': 12, 'Leeds': 13, 'Oxfordshire': 14, 'Washington': 15, 'Luxembourg': 16, 'British Columbia': 17, 'Moscow City': 18, 'Berlin': 19, 'North Carolina': 20, 'Ile-de-France': 21, 'Southern Finland': 22, 'Ontario': 23, 'England': 24, 'Nevada': 25, 'Georgia': 26, 'Colorado': 27, 'Massachusetts': 28, 'Pennsylvania': 29, 'Geneve': 30, 'Goa': 31, 'Tennessee': 32, "Seoul-t'ukpyolsi": 33, 'Sachsen': 34, 'Oklahoma': 35, 'Alabama': 36, 'Central Region': 37, 'Stockholms Lan': 38, 'Ohio': 39, 'Tokyo': 40, 'Zuid-Holland': 41, 'Karnataka': 42, 'Baden-Wurttemberg': 43, 'Surrey': 44, 'Minnesota': 45, 'Maryland': 46, 'New Sou

In [ ]:
# Print some info about our top regions
print(regions.most_common())

[('California', 82659), ('New York', 36883), ('England', 28046), ('Texas', 23375), ('Florida', 19495), ('Ontario', 16690), ('Massachusetts', 15068), ('Illinois', 14298), ('Ile-de-France', 11933), ('Tokyo', 10536), ('Pennsylvania', 10094), ('Maharashtra', 9325), ('Washington', 9143), ('Georgia', 9122), ('Colorado', 8962), ('New Jersey', 8499), ('Virginia', 8101), ('Ohio', 7687), ('Karnataka', 7365), ('Central Region', 7294), ('Bayern', 6972), ('British Columbia', 6936), ('Delhi', 6749), ('Nordrhein-Westfalen', 6718), ('New South Wales', 6622), ('North Carolina', 6548), ('Madrid', 6428), ('Beijing', 6328), ('Michigan', 6109), ('Catalonia', 5836), ('Arizona', 5814), ('Maryland', 5609), ('Quebec', 5582), ('Berlin', 5546), ('Sao Paulo', 5321), ('Minnesota', 5258), ('Tel Aviv', 5091), ('Noord-Holland', 4979), ('Victoria', 4966), ("Seoul-t'ukpyolsi", 4846), ('Guangdong', 4784), ('Baden-Wurttemberg', 4579), ('Alberta', 4536), ('Zurich', 4291), ('Stockholms Lan', 4282), ('Tennessee', 4143), ('S

In [9]:
from scipy.sparse import coo_matrix
import collections

# Load our filtered region dictionary if we've saved it on disk.

row_indices = []
col_indices = []
data_values = []

lookup_index = []
lookup_value = []
used_hashes = set()

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  value = row['region']

  uuid = row['uuid'].strip()
  hash = cityhash.CityHash64(uuid)
  if hash in used_hashes:  # Hash collision!
    print(hash)
    continue          # which shouldn't really happen
  used_hashes.add(hash)

  lookup_value.append(uuid)
  lookup_index.append(hash)

  row_indices.append(i)
  col_indices.append(0)
  data_values.append(hash)

  if value in regions:
    hash = cityhash.CityHash64(row['uuid'])

    row_indices.append(i)
    col_indices.append(region_to_index[value])
    data_values.append(1)

unclustered = coo_matrix((data_values, (row_indices, col_indices)))
print(unclustered.toarray().shape)

Processing  842699  rows.
(842699, 1032)


In [13]:
import scipy.sparse

## Generate company lookup
lookup = pd.DataFrame.from_dict({'uuid':lookup_index, 'hash':lookup_value})
lookup.to_csv('/lookup.csv')

# category_features = pd.DataFrame.sparse.from_spmatrix(joined)
# category_features.to_pickle('/region_features.pkl')

scipy.sparse.save_npz('/region_features.npz', unclustered)


In [11]:
print(unclustered)

  (0, 0)	1.3685534557686295e+19
  (0, 1)	1.0
  (1, 0)	7.640156219293676e+17
  (1, 2)	1.0
  (2, 0)	1.0846552445983457e+19
  (2, 1)	1.0
  (3, 0)	5.087506707876194e+18
  (3, 2)	1.0
  (4, 0)	9.094535307341386e+18
  (4, 2)	1.0
  (5, 0)	1.7418110891307706e+19
  (5, 2)	1.0
  (6, 0)	7.773244737896805e+18
  (6, 2)	1.0
  (7, 0)	1.5263381640365195e+19
  (7, 2)	1.0
  (8, 0)	2.7762472306795356e+18
  (8, 2)	1.0
  (9, 0)	1.7179139454557108e+19
  (9, 2)	1.0
  (10, 0)	3.143841549007344e+18
  (10, 1)	1.0
  (11, 0)	1.0303043534068977e+18
  (11, 2)	1.0
  (12, 0)	2.4974301446054984e+18
  :	:
  (842686, 359)	1.0
  (842687, 0)	1.2078929441060368e+19
  (842687, 2)	1.0
  (842688, 0)	4.0054335165626184e+18
  (842688, 9)	1.0
  (842689, 0)	1.689286265119951e+19
  (842689, 9)	1.0
  (842690, 0)	7.914635032925793e+18
  (842690, 23)	1.0
  (842691, 0)	7.229716827056183e+18
  (842691, 15)	1.0
  (842692, 0)	2.723622163574792e+18
  (842692, 41)	1.0
  (842693, 0)	5.250348886890883e+18
  (842693, 53)	1.0
  (842694, 0)	4.62